In [1]:
from astropy import samp


In [2]:
def wptdReplacement(request,queue):
    samp_name = "unknown"

    if isinstance(request[0], str):
        # To support the old protocol version
        samp_name = request[0]
    else:
        samp_name = request[0]["samp.name"]

    text = \
        """A Web application which declares to be

Name: {}
Origin: {}

is requesting to be registered with the SAMP Hub.
Pay attention that if you permit its registration, such
application will acquire all current user privileges, like
file read/write.

Do you give your consent? [yes|no]""".format(samp_name, request[2])

    print(text)
    #answer = input(">>> ")
    answer="y"
    print(">>> " + answer)
    queue.put(answer.lower() in ["yes", "y"])


In [3]:
import select
import queue
def REPLACE_serve_forever(self):

        while self._is_running:

            try:
                read_ready = select.select([self._server.socket], [], [], 0.01)[0]
            except OSError as exc:
                warnings.warn("Call to select() in SAMPHubServer failed: {0}".format(exc),
                              SAMPWarning)
            else:
                if read_ready:
                    self._server.handle_request()

            if self._web_profile:

                # We now check if there are any connection requests from the
                # web profile, and if so, we initialize the pop-up.
                if self._web_profile_dialog is None:
                    try:
                        request = self._web_profile_requests_queue.get_nowait()
                    except queue.Empty:
                        pass
                    else:
                        #web_profile_text_dialog(request, self._web_profile_requests_result)
                        wptdReplacement(request, self._web_profile_requests_result)

                # We now check for requests over the web profile socket, and we
                # also update the pop-up in case there are any changes.
                try:
                    read_ready = select.select([self._web_profile_server.socket], [], [], 0.01)[0]
                except OSError as exc:
                    warnings.warn("Call to select() in SAMPHubServer failed: {0}".format(exc),
                                  SAMPWarning)
                else:
                    if read_ready:
                        self._web_profile_server.handle_request()

        self._server.server_close()
        if self._web_profile_server is not None:
            self._web_profile_server.server_close()

In [4]:
samp.SAMPHubServer._serve_forever=REPLACE_serve_forever

In [5]:
hub=samp.SAMPHubServer()
hub.start()

INFO: Hub set to run with Web Profile support enabled. [astropy.samp.hub]
INFO: Hub started [astropy.samp.hub]
A Web application which declares to be

Name: TAPHandle
Origin: http://saada.unistra.fr

is requesting to be registered with the SAMP Hub.
Pay attention that if you permit its registration, such
application will acquire all current user privileges, like
file read/write.

Do you give your consent? [yes|no]
>>> y


In [7]:
proxy=samp.SAMPHubProxy()
cli=samp.SAMPClient(proxy)

In [8]:
cli.is_running

False

In [12]:
cli.start()

RuntimeError: threads can only be started once

In [11]:
proxy.connect()

In [13]:
cli.register()

In [16]:
cli.is_registered

True

In [17]:
cli.is_running

True

In [ ]:
cli.bind_receive_call

In [26]:
class Receiver(object):
    def __init__(self, client):
        self.client = client
        self.received = False
    def receive_call(self, private_key, sender_id, msg_id, mtype, params, extra):
        self.params = params
        self.mtype = mtype
        self.received = True
        self.client.reply(msg_id, {"samp.status": "samp.ok", "samp.result": {}})
        print("call received")
    def receive_notification(self, private_key, sender_id, mtype, params, extra):
        self.mtype = mtype
        self.params = params
        self.received = True
        print("notification received")

In [27]:
r=Receiver(cli)

In [28]:
def bindSamp(MType):
    global r
    cli.bind_receive_call(         MType, r.receive_call         )
    cli.bind_receive_notification( MType, r.receive_notification )

In [29]:
bindSamp('qgis.message')
bindSamp('table.load.votable')

notification received


In [31]:
r.params

{'name': 'http://saada.unistra.fr/taphandle/jobresult?NODE=jacobsuni-tap&JOBID=m_pgji&jsessionid=63DAFBCE51DE67B79044221FA8393B1E',
 'table-id': 'http://saada.unistra.fr/taphandle/jobresult?NODE=jacobsuni-tap&JOBID=m_pgji&jsessionid=63DAFBCE51DE67B79044221FA8393B1E',
 'url': 'http://saada.unistra.fr/taphandle/jobresult?NODE=jacobsuni-tap&JOBID=m_pgji&jsessionid=63DAFBCE51DE67B79044221FA8393B1E'}

In [32]:
r.mtype

'table.load.votable'

In [34]:
r.params['url']

'http://saada.unistra.fr/taphandle/jobresult?NODE=jacobsuni-tap&JOBID=m_pgji&jsessionid=63DAFBCE51DE67B79044221FA8393B1E'

In [43]:
import urllib
from astropy.utils.data import download_file # fixes timeout bug
from astropy.table import Table

In [46]:
#f = urllib.request.urlopen(r.params['url'])
#myfile = f.read()  
vot = Table.read(download_file(r.params['url'],timeout=200))
#print(myfile)

In [47]:
vot[1]

granule_uid,granule_gid,obs_id,dataproduct_type,target_name,target_class,time_min,time_max,time_sampling_step_min,time_sampling_step_max,time_exp_min,time_exp_max,spectral_range_min,spectral_range_max,spectral_sampling_step_min,spectral_sampling_step_max,spectral_resolution_min,spectral_resolution_max,c1min,c1max,c2min,c2max,c3min,c3max,s_region,c1_resol_min,c1_resol_max,c2_resol_min,c2_resol_max,c3_resol_min,c3_resol_max,spatial_frame_type,incidence_min,incidence_max,emergence_min,emergence_max,phase_min,phase_max,instrument_host_name,instrument_name,measurement_type,processing_level,creation_date,modification_date,release_date,service_title,access_url,access_format,access_estsize,thumbnail_url,bib_reference,solar_longitude_min,solar_longitude_max,sensor_id,image_width,image_height,external_link,spatial_coordinate_description,wavelengths
,,,,,,d,d,s,s,s,s,Hz,Hz,Hz,Hz,Hz,Hz,deg,deg,deg,deg,m,m,,deg,deg,deg,deg,m,m,,deg,deg,deg,deg,deg,deg,,,,,,,,,,,kbyte,,,deg,deg,,pix,pix,,,nm
object,object,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,object,object,object,int32,object,object,object,object,object,object,int32,object,object,float64,float64,object,int32,int32,object,object,object
FRT00008306_07_IF163L_TRR3,FRT,00008306,sc,Mars,planet,2454387.15795,2454387.15915,--,--,--,--,76151305120910.4,299403233796065.0,126910768664.078,1945816521669.06,290151792525.314,2990145149266.6,323.008,323.261,-34.646,-34.492,--,--,Polygon UNKNOWNFrame 323.261 -34.629 323.141 -34.635 323.081 -34.641 323.022 -34.646 323.032 -34.612 323.036 -34.592 323.027 -34.566 323.008 -34.526 323.069 -34.516 323.189 -34.498 323.249 -34.492 323.239 -34.546 323.236 -34.573 323.243 -34.596,0.000335543766578306,0.000335543766578306,0.000333333333333341,0.000333333333333341,--,--,body,330.134752,330.134752,1.98791170120239,1.98791170120239,39.3845405578613,39.3845405578613,MRO,CRISM,phys.luminosity;phys.angArea;em.wl,3,2016-01-01T00:00:00,2018-02-08T00:00:00,2016-01-01T00:00:00,CRISM,http://access.planetserver.eu/rasdaman/ows?&SERVICE=WCS&VERSION=2.0.1&REQUEST=GetCoverage&COVERAGEID=frt00008306_07_if163l_trr3&FORMAT=image/tiff,application/x-geotiff,614485,"http://access.planetserver.eu/rasdaman/ows?service=WCS&version=2.0.1&request=ProcessCoverages&query=for%20data%20in%20%28%20frt00008306_07_if163l_trr3%20%29%20return%20encode(%20{%20red:%20(int)((int)(255%20/%20(max(%20data.band_233)%20-%20min(data.band_233)))%20*%20(data.band_233%20-%20min(data.band_233)));%20green:%20(int)((int)(255%20/%20(max(%20data.band_78)%20-%20min(data.band_78)))%20*%20(data.band_78%20-%20min(data.band_78)));%20blue:%20(int)((int)(255%20/%20(max(%20data.band_13)%20-%20min(data.band_13)))%20*%20(data.band_13%20-%20min(data.band_13)))%20;%20alpha:%20(int)((data.band_100%20%3E%200)%20*%20255)},%20%22png%22,%20%22nodata=65535%22)",10.1029/2006JE002682,330.134752,330.134752,L,754,462,http://aux1.epn-vespa.jacobs-university.de/subGranule3/index.html?callback=frt00008306_07_if163l_trr3,+proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +a=3396190 +b=3396190 +units=m +no_defs,1.00135;1.0079;1.01445;1.021;1.02755;1.0341;1.04065;1.0472;1.05375;1.0603;1.06685;1.07341;1.07996;1.08651;1.09307;1.09962;1.10617;1.11273;1.11928;1.12584;1.13239;1.13895;1.14551;1.15206;1.15862;1.16518;1.17173;1.17829;1.18485;1.19141;1.19797;1.20453;1.21109;1.21765;1.22421;1.23077;1.23733;1.24389;1.25045;1.25701;1.26357;1.27014;1.2767;1.28326;1.28983;1.29639;1.30295;1.30952;1.31608;1.32265;1.32921;1.33578;1.34234;1.34891;1.35548;1.36205;1.36861;1.37518;1.38175;1.38832;1.39489;1.40145;1.40802;1.41459;1.42116;1.42773;1.43431;1.44088;1.44745;1.45402;1.46059;1.46716;1.47374;1.48031;1.48688;1.49346;1.50003;1.50661;1.51318;1.51976;1.52633;1.53291;1.53948;1.54606;1.55264;1.55921;1.56579;1.57237;1.57895;